In [1]:
print("hello")

hello


In [2]:
%pwd

"c:\\Arjun's Work\\MediVerse\\research"

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

"c:\\Arjun's Work\\MediVerse"

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [6]:
def load_documents(data):
  loader = DirectoryLoader(data, glob="**/*.pdf", loader_cls=PyPDFLoader)
  documents = loader.load()
  return documents

In [7]:
extracted_data = load_documents(data = "Docs/")

Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 47 0 (offset 0)
Ignoring wrong pointing object 64 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 114 0 (offset 0)
Ignoring wrong pointing object 173 0 (offset 0)
Ignoring wrong pointing object 181 0 (offset 0)


In [23]:
def text_split(extracted_data):
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=2000,
      chunk_overlap=500,
      length_function=len,
  )
  texts = text_splitter.split_documents(extracted_data)
  return texts

In [24]:
text_chunks = text_split(extracted_data)
print("Length of text chunks: ", len(text_chunks))

Length of text chunks:  1251


In [25]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_huggingface_embeddings():
  embeddings = HuggingFaceEmbeddings(
      model_name="sentence-transformers/all-MiniLM-L6-v2",
  )
  return embeddings

In [26]:
embeddings = download_huggingface_embeddings()

In [27]:
query_result = embeddings.embed_query("What is the capital of France?")
print("Length: ", len(query_result))

Length:  384


In [28]:
query_result

[0.08204811811447144,
 0.03605549782514572,
 -0.0038928815629333258,
 -0.004881042055785656,
 0.025651127099990845,
 -0.057143475860357285,
 0.012191563844680786,
 0.00467892037704587,
 0.0349498875439167,
 -0.02242197096347809,
 -0.00800524465739727,
 -0.10935357213020325,
 0.022724736481904984,
 -0.029320916160941124,
 -0.04352208599448204,
 -0.1202412024140358,
 -0.0008486119331791997,
 -0.018150130286812782,
 0.056129518896341324,
 0.0030852793715894222,
 0.002336376579478383,
 -0.016839278861880302,
 0.06362469494342804,
 -0.023660195991396904,
 0.031493544578552246,
 -0.034797921776771545,
 -0.020548837259411812,
 -0.002790994942188263,
 -0.01103801280260086,
 -0.036126717925071716,
 0.05414106324315071,
 -0.03661714866757393,
 -0.025008678436279297,
 -0.038170378655195236,
 -0.0496036522090435,
 -0.015148064121603966,
 0.02131502889096737,
 -0.012740440666675568,
 0.07670093327760696,
 0.04435573145747185,
 -0.010834850370883942,
 -0.029760019853711128,
 -0.01697046123445034,
 -

In [29]:
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

In [31]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)


index_name = "fitsync"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': '1bdd53952e755d0e0e876af1fc44b541', 'date': 'Sun, 11 May 2025 18:45:18 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [32]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [34]:
def split_large_chunks(chunks, max_tokens=512):
    from langchain.text_splitter import RecursiveCharacterTextSplitter

    splitter = RecursiveCharacterTextSplitter(chunk_size=max_tokens, chunk_overlap=50)
    small_chunks = []
    for chunk in chunks:
        small_chunks.extend(splitter.split_text(chunk.page_content))
    return small_chunks


In [35]:
from langchain_pinecone import PineconeVectorStore

small_chunks = split_large_chunks(text_chunks)
docsearch = PineconeVectorStore.from_texts(
    small_chunks,
    embedding=embeddings,
    index_name=index_name,
    batch_size=16, 
)

